# Soh estimation experimentation of Ford vehicles
Based on the file `ford_eda.ipynb` we have seen that we could use the `charging.battery_energy` to estimate the SoH.  

All those result are based on what has been analysed in `ford_eda.ipynb`

We added one filter on the SoC for the SoH calculation, a seocnd one on the charge/discharge could also be added.

In [ ]:
! mkdir -p data_cache/

## Imports

In [ ]:
from datetime import datetime as DT
from datetime import timedelta as TD
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import numpy as np
from rich import print
from rich.progress import track
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px
from scipy import stats

from core.s3_utils import S3_Bucket
from core.constants import *
from core.ev_models_info import models_info
from transform.fleet_info.ayvens_fleet_info import fleet_info
from transform.raw_tss.main import get_raw_tss

## Setup

### Data extraction

In [ ]:

COLS_TO_CPY_FROM_FLEET_INFO = [
    "make",
    "model",
    "version",
    "dummy_soh_maker_offset",
    "dummy_soh_model_offset",
    "dummy_soh_model_slope",
    "dummy_soh_vehicle_offset",
    "capacity",
]

RENAME_COLS_DICT = {
    "date_of_value": "date",
    "diagnostics.odometer": "odometer",
    "odometer.value": "odometer",
    "diagnostics.odometer": "odometer",
    "mileage": "odometer",
    "charging.battery_energy": "battery_energy",
    "charging.estimated_range": "estimated_range",
    "charging.battery_level": "soc",
    "soc_hv_header": "soc",
}

COLS_TO_KEEP = [
    "date",
    "soc",
    "odometer",
    "estimated_range",
    "battery_energy",
    "soc",
    "vin",
]

COL_DTYPES = {
    "soc": "float",
    "odometer": "float",
    # "estimated_range": "float",
    "battery_energy": "float",
    "soc": "float",
    "dummy_soh_maker_offset": "float",
    "dummy_soh_model_offset": "float",
    "dummy_soh_model_slope": "float",
    "dummy_soh_vehicle_offset": "float",
    "dummy_soh_offset": "float",
    "vin": "string",
    "capacity": "float",
}

# Add model and model version columns
raw_tss = get_raw_tss(brand="ford").rename(columns=RENAME_COLS_DICT)
cols_to_drop = raw_tss.columns[~raw_tss.columns.isin(COLS_TO_KEEP)]
raw_tss = raw_tss.drop(columns=cols_to_drop)
raw_tss[COLS_TO_CPY_FROM_FLEET_INFO] = fleet_info.loc[raw_tss["vin"], COLS_TO_CPY_FROM_FLEET_INFO].values
raw_tss:DF = raw_tss.eval("dummy_soh_offset = dummy_soh_maker_offset + dummy_soh_model_offset + dummy_soh_vehicle_offset")



In [ ]:
raw_tss.columns

In [ ]:
tss = (
    raw_tss
    .astype(COL_DTYPES)
    .sort_values(by=["make", "vin", "date"])
)


In [ ]:
tss["date"] = pd.to_datetime(tss["date"], format="mixed").dt.tz_localize(None)
tss["registration_date"] = pd.to_datetime(fleet_info.loc[tss["vin"], "registration_date"].values, format="mixed")
tss["age_in_years"] = tss.eval("date - registration_date").dt.days.div(365)

In [ ]:
tss["age_in_years"].sort_values(ascending=False)

## SOH

Let's visualize the soh estimation.

In [ ]:
ford_tss:DF = (
    tss
    .query("make == 'ford'")
    .query("soc > 0.5")
    .eval("soh = battery_energy / soc / capacity * 100")
)

ford_vehicles = (
    ford_tss
    .groupby("vin")
    .agg({
        "soh": "mean",
        "odometer": "last",
        "model": "first",
        "date": "last",
        "age_in_years": "last",
        "version": "last",
    })
    .reset_index(drop=False)
)

In [ ]:
fig = px.scatter(
    ford_vehicles,
    x="odometer",
    y="soh",
    color="model",
    height=600,
    title="Average State-of-Health (SoH) vs Mileage",
    trendline="ols",
    # trendline_scope="overall",
    hover_data=["vin"]
)
fig.update_layout(
    xaxis_title="Latest mileage (km)",
    yaxis_title="SoH (%)",
    legend_title="Model",
)
fig.update_traces(line=dict(color='black', dash='dash'))

fig.show()
fig.to_html("data_cache/soh_plot.html")

In [ ]:
ford_vehicles["soh"]

-> There is no trend of battery degrading over the mileage.  
Which make us say that the SoH is not a good estimator for the battery degradation.  